## Homework6
時系列データを含むデータベースファイル (data.sqlite3) に含まれるデータについて、goods_genre_id ごとに price の平均値を求めてください。時系列データには goods_genre_id の情報は含まれていないので、複数のテーブルを組み合わせて処理する必要があります。  
データの詳細は以下を参照してください。

### データ一覧
今回与えられるデータは data.sqlite3 という sqlite3 形式のデータです。
以下の 2 つのテーブルが含まれています。
* data ... 時系列データ
* goods ... 商品に関する補足情報

### カラム情報一覧
データに含まれるカラムの情報は以下です。
* index ... (店舗, 商品) の各組み合わせに対して割り振られるID
* store_id ... 店舗ID
* goods_id ... 商品ID
* goods_genre_id ... 商品カテゴリID
* units_sold_day ... 商品の一日の売り上げ個数
* units_sold_month ... 商品のひと月の売り上げ個数
* price ... 商品の現在の値段
* yy-mm-dd ... 日付 (yy/mm/dd形式)
* num_month ... 便宜上各月にわりふられた値 (平成25年1月=1, 平成25年2月=2,...,平成27年9月=33)
* goods_name ... 商品名
* store_name ... 店舗名
* goods_genre_name ... 商品カテゴリ名

### 注意事項
- データ処理はSQLiteでの操作を推奨します。補足資料「SQLite_manual.pdf」「SQLite_manual.ipynb」を参照ください。
- SQLiteのインストール/ダウンロードが上手くいかない場合、python上での操作も可能です。補足資料「csv_export_python.pdf」「csv_export_python.ipynb」を参照ください。
- 一部の処理に時間がかかる場合があります。

### 提出/採点
回答は**Omnicampus**上で提出してください。提出するデータの形式は以下のような**csvファイル**となります。
 
goods_genre_id|avg_price
---|---
1|25000
3|26500
4|25000
…|…
196|20000
200|27000


提出ファイルがこの形式に準じていなかった場合にはスコアが1000となるので、その際はファイル形式を確認しましょう。  

In [1]:
import sqlite3
import csv

In [2]:
# Google Colaboratoryで読み込み（ドライブのマウント）
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# データベースファイルのパスを渡す
dbpath = '/content/drive/MyDrive/Colab Notebooks/week7/data.sqlite3'
conn = sqlite3.connect(dbpath)
cur = conn.cursor()

In [4]:
# テーブル名の確認（2種類；data, goods）
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('data',)
('goods',)


In [5]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)',)
('CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)',)


In [6]:
cur.execute('select * from goods')
print(cur.fetchone()) #1レコード目の取得
print(cur.fetchone()) #2レコード目の取得

('! POWER IN glamor (PLAST.) D', '1', '41')
('! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]', '2', '77')


In [7]:
#テーブルから全ての列を取得（data）
cur.execute('select * from data')
with open('out_data.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [8]:
#テーブルから全ての列を取得(goods)
cur.execute('select * from goods')
with open('out_goods.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [9]:
# 処理を確定
conn.commit()
cur.close()
conn.close()

In [10]:
#---------------------------
import pandas as pd

#格納
data = pd.read_csv('out_data.csv', sep=',')
data
goods = pd.read_csv('out_goods.csv', sep=',')
goods

,goods_name,goods_id,goods_genre_id
0,! POWER IN glamor (PLAST.) D,1,41
1,! ABBYY FineReader 12 Professional Edition Ful...,2,77
2,*** In the glory (UNV) D,3,41
3,*** BLUE WAVE (Univ) D,4,41
4,*** BOX (GLASS) D,5,41
...,...,...,...
22165,"Nuclear titbit 2 [PC, Digital Version]",22166,32
22166,Language 1C queries: Enterprises [Digital Vers...,22167,55
22167,1C query language: Enterprise 8 (+ CD). Khrust...,22168,50
22168,Egg for Little Inu,22169,63


In [11]:
# データのマージ（完全外部結合）
merge = pd.merge(data, goods, how = 'outer')
merge

,yy_mm_dd,num_month,store_id,goods_id,price,units_sold_day,goods_name,goods_genre_id
0,25-01-29,-33.0,43.0,17718,44943.0,2.0,Receiving cash for 1C-line,80
1,25-01-08,-33.0,32.0,8648,14949.0,1.0,Barbie as the Princess and pop stars (region),41
2,25-01-16,-33.0,53.0,10168,30158.0,1.0,Vysotsky Vladimir theatrical sketch on Tagansk...,60
3,25-01-23,-33.0,31.0,1005,9922.0,1.0,"3D Action Puzzle ""Technique"" racing machine",68
4,25-01-12,-33.0,27.0,19861,14988.0,0.0,Three from Buttermilk (Region),41
...,...,...,...,...,...,...,...,...
22374,NaN,NaN,NaN,22166,NaN,NaN,"Nuclear titbit 2 [PC, Digital Version]",32
22375,NaN,NaN,NaN,22167,NaN,NaN,Language 1C queries: Enterprises [Digital Vers...,55
22376,NaN,NaN,NaN,22168,NaN,NaN,1C query language: Enterprise 8 (+ CD). Khrust...,50
22377,NaN,NaN,NaN,22169,NaN,NaN,Egg for Little Inu,63


In [12]:
#平均
out = merge.groupby('goods_genre_id', as_index = False)['price'].mean()
out

,goods_genre_id,price
0,1,NaN
1,2,NaN
2,3,168482.4
3,4,47476.5
4,5,129025.0
...,...,...
79,80,44943.0
80,81,NaN
81,82,6410.0
82,83,1911.0


In [13]:
# 列名変更
out_1 = out.rename(columns={'price': 'avg_price'})
out_1

,goods_genre_id,avg_price
0,1,NaN
1,2,NaN
2,3,168482.4
3,4,47476.5
4,5,129025.0
...,...,...
79,80,44943.0
80,81,NaN
81,82,6410.0
82,83,1911.0


In [14]:
#csv出力
out_1.to_csv('out_1.csv',index=False)
from google.colab import files
files.download('out_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>